In [ ]:
#!/usr/bin/env python3
"""
 
"""

import os
import shutil
import json
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# Configuration
BASE_DIR = os.path.abspath(r"C:\Users\kosmo\pycode\Iqvia_process")
CHEMIN_SOURCE = os.path.join(BASE_DIR, "ProcessEx")
CHEMIN_DEPOTS = os.path.join(BASE_DIR, "Depots")

# Initialiser le LLM
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("⚠️ ATTENTION: Clé API non trouvée dans les variables d'environnement")
    api_key = "votre_clé_api_openai_ici"  # À remplacer par votre vraie clé

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=api_key
)

def detection_mots_cles_medicaux(titre: str) -> dict:
    """Détection fallback par mots-clés"""
    titre_lower = titre.lower()
    
    termes_medicaux = {
        'cancer': 'oncologie', 'tumeur': 'oncologie', 'oncologie': 'oncologie',
        'prostate': 'oncologie', 'sein': 'oncologie', 'poumon': 'oncologie',
        'diabète': 'diabetologie', 'diabete': 'diabetologie', 'glycémie': 'diabetologie',
        'cardiaque': 'cardiologie', 'coeur': 'cardiologie', 'cœur': 'cardiologie',
        'hypertension': 'cardiologie', 'infarctus': 'cardiologie',
        'alzheimer': 'neurologie', 'parkinson': 'neurologie', 'neurologie': 'neurologie',
        'asthme': 'pneumologie', 'pneumonie': 'pneumologie', 'bronchite': 'pneumologie',
        'eczéma': 'dermatologie', 'eczema': 'dermatologie', 'psoriasis': 'dermatologie'
    }
    
    maladies_detectees = [terme for terme in termes_medicaux.keys() if terme in titre_lower]
    
    if maladies_detectees:
        categorie_finale = termes_medicaux[maladies_detectees[0]]
        confiance = "haute" if len(maladies_detectees) > 1 else "moyenne"
        return {
            "contient_maladie": True,
            "maladies_detectees": maladies_detectees,
            "categorie_medicale": categorie_finale,
            "confiance": confiance,
            "titre_normalise": titre
        }
    
    return {
        "contient_maladie": False,
        "maladies_detectees": [],
        "categorie_medicale": "aucune",
        "confiance": "faible",
        "titre_normalise": titre
    }

def analyser_titre_avec_llm(titre: str) -> dict:
    """Analyse avec LLM - même prompt que LangGraph"""
    try:
        analysis_prompt_template = ChatPromptTemplate.from_messages([
            ("system", "Vous êtes un expert en terminologie médicale. Analysez les titres de documents médicaux avec précision."),
            ("human", """
Analysez ce titre de fichier médical: "{titre}"

RÈGLES PRIORITAIRES D'EXCLUSION:
- Exclure tous les documents qui ne sont PAS des propositions médicales
- Exclure les suivis, études, rapports, bilans, analyses de marché
- Exclure les documents techniques, commerciaux, contractuels
- Exclure les MSA, SOW, PO, contrats, accords

APRÈS vérification des exclusions, recherchez ces termes de propositions médicales:
proposition, proposal, offre, devis, cotation, tarification, budget, estimation, soumission, candidature

EXEMPLES D'ANALYSE:

EXEMPLES DE FAUX POSITIFS supplémentaires à ajouter :
Le faux positif a la forme (termes de proposition) mais pas le fond (vraie proposition médicale)

Faux positifs techniques/commerciaux :
- "3181844 _IQVIA pour ABBOTT DC_Suivi national du FSL2 par segment de pat diabétiques 2025" → contient "suivi" mais c'est un rapport, pas une proposition → aucune
- "3252322 - Suivi Xponent std cancer de la prostate 2025" → contient "suivi" mais c'est un rapport → aucune
- "3270205_IQVIA pour ABBVIE_Channel Dynamics_Bilan Marché DLBCL_2025" → contient "bilan" mais c'est une analyse → aucune

Faux positifs contractuels :
- "MSA_ABBOTT PT 60 days" → Master Service Agreement → aucune
- "PO4202456147" → Purchase Order → aucune
- "Complete_with_Docusign_IQVIA_SOW_Accord_Heal" → document contractuel → aucune

Exemples POSITIFS à classer :

Propositions oncologie :
- "3225553 - Proposition commerciale pour ABBVIE - LTD Immunologie oncologie" → contient: proposition, oncologie → proposition_oncologie
- "Devis étude cancer prostate IQVIA 2025" → contient: devis, cancer, prostate → proposition_oncologie
- "Offre DLBCL lymphome suivi patients" → contient: offre, DLBCL → proposition_oncologie

Propositions dermatologie :
- "Proposition étude dermatite atopique ABBOTT" → contient: proposition, dermatite → proposition_dermatologie
- "Cotation Vistabel market access" → contient: cotation, Vistabel → proposition_dermatologie

Propositions neurologie :
- "Budget étude Parkinson Xponent 2025" → contient: budget, Parkinson → proposition_neurologie
- "Devis Alzheimer patient journey" → contient: devis, Alzheimer → proposition_neurologie

Propositions cardiologie :
- "Estimation coût étude hypertension" → contient: estimation, hypertension → proposition_cardiologie

La règle clé est : le document doit être une vraie proposition/offre commerciale concernant une pathologie/domaine médical, pas un rapport ou suivi mentionnant des termes de proposition.

Répondez UNIQUEMENT en JSON valide:
{
    "est_proposition_medicale": true/false,
    "termes_proposition_detectes": ["terme1", "terme2"],
    "domaine_medical": "oncologie/cardiologie/neurologie/diabetologie/pneumologie/dermatologie/immunologie/ophtalmologie/autre/aucun",
    "confiance": "haute/moyenne/faible",
    "titre_normalise": "{titre}"
}
            """)
        ])
        
        prompt_value = analysis_prompt_template.invoke({"titre": titre})
        response = llm.invoke(prompt_value.to_messages())
        content = response.content.strip()
        
        # Nettoyer le JSON
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0]
        elif "```" in content:
            content = content.split("```")[1].split("```")[0]
        
        return json.loads(content.strip())
        
    except Exception as e:
        print(f"❌ Erreur LLM pour '{titre[:30]}...': {e}")
        return detection_mots_cles_medicaux(titre)

def rechercher_fichiers():
    """Phase 1: Recherche tous les fichiers dans ProcessEx"""
    print("🔍 PHASE 1: Recherche des fichiers...")
    print("-" * 50)
    
    if not os.path.exists(CHEMIN_SOURCE):
        print(f"❌ Erreur: {CHEMIN_SOURCE} n'existe pas")
        return []
    
    fichiers_trouves = []
    dossiers_stats = {}
    
    for item in os.listdir(CHEMIN_SOURCE):
        item_path = os.path.join(CHEMIN_SOURCE, item)
        if os.path.isdir(item_path):
            print(f"📂 Analyse du dossier: {item}")
            count = 0
            
            for root, dirs, files in os.walk(item_path):
                for file in files:
                    if file.lower().endswith(('.ppt', '.pptx', '.pdf')):
                        fichiers_trouves.append(os.path.join(root, file))
                        count += 1
            
            dossiers_stats[item] = count
            print(f"  ✅ {count} fichier(s) trouvé(s)")
    
    print(f"\n📊 TOTAL: {len(fichiers_trouves)} fichiers dans {len(dossiers_stats)} dossiers")
    for dossier, count in dossiers_stats.items():
        print(f"  • {dossier}: {count}")
    
    return fichiers_trouves

def copier_fichiers(fichiers_trouves):
    """Phase 2: Copie vers Depots"""
    print(f"\n📋 PHASE 2: Copie vers {CHEMIN_DEPOTS}...")
    print("-" * 50)
    
    # Créer le dossier Depots
    os.makedirs(CHEMIN_DEPOTS, exist_ok=True)
    print(f"✅ Dossier Depots prêt")
    
    copied = 0
    for file_path in fichiers_trouves:
        file_name = os.path.basename(file_path)
        destination = os.path.join(CHEMIN_DEPOTS, file_name)
        
        try:
            shutil.copy(file_path, destination)
            copied += 1
        except Exception as e:
            print(f"❌ Erreur copie {file_name}: {e}")
    
    print(f"✅ {copied}/{len(fichiers_trouves)} fichiers copiés")
    return copied

def analyser_et_classer():
    """Phase 3: Analyse LLM et classification"""
    print(f"\n🤖 PHASE 3: Analyse IA et classification...")
    print("-" * 50)
    
    # Mode LLM ou fallback
    use_llm = api_key != "votre_clé_api_openai_ici"
    print(f"Mode: {'🔥 LLM' if use_llm else '🔧 Mots-clés'}")
    
    fichiers_a_analyser = [f for f in os.listdir(CHEMIN_DEPOTS) 
                          if f.lower().endswith(('.ppt', '.pptx', '.pdf'))]
    
    analyses = []
    medicaux = 0
    
    for i, file_name in enumerate(fichiers_a_analyser, 1):
        titre = os.path.splitext(file_name)[0]
        print(f"[{i:2d}/{len(fichiers_a_analyser)}] {file_name[:60]}...")
        
        # Analyser
        if use_llm:
            resultat = analyser_titre_avec_llm(titre)
        else:
            resultat = detection_mots_cles_medicaux(titre)
        
        analyses.append({
            'fichier': file_name,
            'maladies_detectees': resultat['maladies_detectees'],
            'categorie_medicale': resultat['categorie_medicale'],
            'confiance': resultat['confiance'],
            'contient_maladie': resultat['contient_maladie']
        })
        
        # Classer le fichier
        ancien_chemin = os.path.join(CHEMIN_DEPOTS, file_name)
        
        if resultat["contient_maladie"]:
            # Dossier médical
            categorie = resultat["categorie_medicale"]
            dossier_medical = os.path.join(CHEMIN_DEPOTS, f"MEDICAL_{categorie.upper()}")
            os.makedirs(dossier_medical, exist_ok=True)
            nouveau_chemin = os.path.join(dossier_medical, file_name)
            medicaux += 1
            print(f"     → MEDICAL_{categorie.upper()}/ ({'|'.join(resultat['maladies_detectees'])})")
        else:
            # Dossier autres
            dossier_autres = os.path.join(CHEMIN_DEPOTS, "AUTRES")
            os.makedirs(dossier_autres, exist_ok=True)
            nouveau_chemin = os.path.join(dossier_autres, file_name)
            print(f"     → AUTRES/")
        
        # Déplacer
        try:
            shutil.move(ancien_chemin, nouveau_chemin)
        except Exception as e:
            print(f"     ❌ Erreur déplacement: {e}")
    
    print(f"\n📊 RÉSULTATS: {medicaux}/{len(analyses)} fichiers médicaux classés")
    return analyses

def generer_rapport(analyses):
    """Phase 4: Génération du rapport"""
    print(f"\n📝 PHASE 4: Génération du rapport...")
    print("-" * 50)
    
    fichiers_medicaux = sum(1 for a in analyses if a['contient_maladie'])
    categories = {}
    for a in analyses:
        if a['contient_maladie']:
            cat = a['categorie_medicale']
            categories[cat] = categories.get(cat, 0) + 1
    
    rapport = f"""# 📊 Rapport d'Analyse de Fichiers Médicaux (Script Simple)

**Date d'analyse :** {datetime.now().strftime("%d/%m/%Y à %H:%M:%S")}  
**Dossier analysé :** `{CHEMIN_DEPOTS}`
**Méthode :** Script Python direct (sans LangGraph)

---

## 📈 Résumé Statistique

| Métrique | Valeur |
|----------|--------|
| **Fichiers analysés** | {len(analyses)} |
| **Fichiers médicaux** | {fichiers_medicaux} |
| **Fichiers non-médicaux** | {len(analyses) - fichiers_medicaux} |
| **Catégories détectées** | {len(categories)} |

---

## 🏥 Répartition par Spécialités

"""
    
    if categories:
        for cat, count in sorted(categories.items()):
            pct = (count / fichiers_medicaux * 100) if fichiers_medicaux > 0 else 0
            rapport += f"- **{cat.title()}** : {count} fichier(s) ({pct:.1f}%)\n"
    else:
        rapport += "Aucune catégorie médicale détectée.\n"
    
    rapport += "\n---\n\n## 📋 Détail par Fichier\n\n"
    
    for analyse in analyses:
        dossier = f"MEDICAL_{analyse['categorie_medicale'].upper()}" if analyse['contient_maladie'] else "AUTRES"
        rapport += f"""### 📄 {analyse['fichier']}
- **Maladies :** {', '.join(analyse['maladies_detectees']) if analyse['maladies_detectees'] else 'Aucune'}
- **Catégorie :** {analyse['categorie_medicale']}
- **Dossier :** `{dossier}/`

"""
    
    rapport += f"""---

## ⚙️ Technologie

- **Script :** Python simple (90% moins de code que LangGraph)
- **LLM :** ChatOpenAI GPT-4o-mini 
- **Fallback :** Détection par mots-clés
- **Performance :** {len(analyses)} fichiers traités

---

*Rapport généré par script Python simple - Preuve que LangGraph était de la sur-ingénierie ! 😉*
"""
    
    # Sauvegarder
    chemin_rapport = os.path.join(CHEMIN_DEPOTS, "RAPPORT_SIMPLE.md")
    with open(chemin_rapport, 'w', encoding='utf-8') as f:
        f.write(rapport)
    
    print(f"✅ Rapport sauvé: {chemin_rapport}")

def main():
    """Fonction principale - workflow simple"""
    print("🚀 CLASSIFICATEUR MÉDICAL SIMPLE")
    print("=" * 60)
    print(f"Source: {CHEMIN_SOURCE}")
    print(f"Destination: {CHEMIN_DEPOTS}")
    print("=" * 60)
    
    # Workflow en 4 étapes simples
    fichiers = rechercher_fichiers()
    if not fichiers:
        print("❌ Aucun fichier trouvé - Arrêt")
        return
    
    copier_fichiers(fichiers)
    analyses = analyser_et_classer()
    generer_rapport(analyses)
    
    print("\n" + "🎉" * 20)
    print("✅ TERMINÉ - Même résultat qu'avec LangGraph !")
    print("💡 90% moins de code, même performance")
    print("🎉" * 20)

if __name__ == "__main__":
    main()